# RL 훈련 작업을 위한 하이퍼파라메터 조정

이 노트북은 SageMaker의 Roboschool 환경 하에서 자동 모델 튜닝 기능을 사용하여 RL 모델의 훈련을 최적화하는 방법을 보여줍니다. SageMaker 자동 모델 튜닝에 사용된 베이지안 하이퍼파라미터 최적화 알고리즘은 안정적이고 객관적인 기능을 구현합니다. 즉, 동일한 구성으로 동일한 훈련 작업을 여러 번 실행하면 동일한 결과를 얻을 수 있습니다. 하지만, 일부 RL 훈련 프로세스는 비결정적이기(non-deterministic) 때문에 동일한 구성이 때때로 제대로 수행되고 때로는 훈련에 실패합니다. 이러한 환경은 자동 모델 튜너에서 잘 작동하지 않습니다. 그러나, Roboschool 환경은 상당히 안정적이며 동일한 하이퍼파라미터가 주어지면 비슷한 성능을 발휘합니다. 이 노트북 예제를 통해 Roboscool 환경에서의 하이퍼파라메터 튜너가 안정적으로 작동하는 것을 볼 수 있습니다.

## 해결해야 할 Roboschool 문제 선택

Roboschool은 로봇 시스템에 대한 RL 정책(policy)을 훈련시키는 데 일반적으로 사용되는 [오픈 소스](https://github.com/openai/roboschool/tree/master/roboschool) 물리 시뮬레이터입니다. Roboschool은 다양한 로봇 문제에 해당하는 [다양한](https://github.com/openai/roboschool/blob/master/roboschool/__init__.py) gym 환경을 정의합니다. 아래는 다양한 난이도 중 몇 가지를 보여줍니다.

- **Reacher (쉬움)** - 2개의 조인트만 있는 매우 간단한 로봇이 목표물에 도달합니다.
- **호퍼 (중간)** - 한쪽 다리와 발이 달린 간단한 로봇이 트랙을 뛰어 내리는 법을 배웁니다.
- **휴머노이드 (어려움)** - 두 개의 팔, 두 개의 다리 등이 있는 복잡한 3D 로봇은 넘어지지 않고 균형을 잡은 다음 트랙에서 달리는 법을 배웁니다.

간단한 문제들은 적은 계산 리소스 상에서 더 빨리 훈련됩니다. 물론 더 복잡한 문제들은 훈련이 느리지만 더 재미있습니다.

In [1]:
# Uncomment the problem to work on
#roboschool_problem = 'reacher'
roboschool_problem = 'hopper'
#roboschool_problem = 'humanoid'

## 전제 조건(Pre-requisites)

### 라이브러리 임포트

시작하기 위해, 필요한 Python 라이브러리를 가져와서 권한 및 구성을 위한 몇 가지 전제 조건으로 환경을 설정합니다.

In [2]:
import sagemaker
import boto3
import sys
import os
import glob
import re
import subprocess
from IPython.display import HTML
import time
from time import gmtime, strftime
sys.path.append("common")
from misc import get_execution_role, wait_for_s3_object
from docker_utils import build_and_push_docker_image
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework

### S3 버킷 설정

체크포인트(checkpoint) 및 메타데이터에 사용하려는 S3 버킷에 대한 연결 및 인증을 설정합니다.

In [3]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

S3 bucket path: s3://sagemaker-us-west-2-143656149352/


### 변수 설정

훈련 작업의 작업 접두사(job prefix)와 *컨테이너의 이미지 경로(BYOC 인 경우에만)와 같은 변수*를 정의합니다.

In [4]:
# create a descriptive job name 
job_name_prefix = 'tune-'+roboschool_problem

### 튜닝에 사용할 리소스 구성

튜닝 작업은 로컬 모드가 아닌 호스팅된 SageMaker에서 수행해야 합니다. 따라서 인스턴스 유형을 선택하고 `max_jobs`와 `max_parallel_jobs`도 지정해야 합니다.

In [5]:
# Note that Tuning cannot happen in local mode.
instance_type = "ml.m4.xlarge"

# Pick the total number of training jobs to run in this tuning job
max_jobs = 50

# How many jobs should run at a time.  Higher numbers here mean the tuning job runs much faster,
# while lower numbers can sometimes get better results
max_parallel_jobs = 5

### IAM 역할 생성

SageMaker 노트북 `role = sagemaker.get_execution_role()`을 실행할 때 실행 역할(execution role)을 얻거나 로컬 시스템에서 실행할 때 utils 메소드 `role = get_execution_role()`을 사용하여 실행 역할을 작성하세요.

In [6]:
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()

print("Using IAM role arn: {}".format(role))

Using IAM role arn: arn:aws:iam::143656149352:role/service-role/AmazonSageMaker-ExecutionRole-20200323T143836


## 도커 컨테이너 빌드

Roboschool이 설치된 사용자 정의 도커 컨테이너를 빌드해야 합니다. 컨테이너 빌드 작업은 아래 과정을 거쳐 처리됩니다.

1. 기본 컨테이너 이미지 가져오기
2. Roboschool 및 의존성 패키지 설치
3. 새 컨테이너 이미지를 ECR에 업로드

인터넷 연결이 느린 컴퓨터에서 실행 중인 경우, 이 단계에서 시간이 오래 걸릴 수 있습니다. 노트북 인스턴스가 SageMaker 또는 EC2 인 경우 인스턴스 유형에 따라 3-10 분이 걸립니다.

In [7]:
%%time

cpu_or_gpu = 'gpu' if instance_type.startswith('ml.p') else 'cpu'
repository_short_name = "sagemaker-roboschool-ray-%s" % cpu_or_gpu
docker_build_args = {
    'CPU_OR_GPU': cpu_or_gpu, 
    'AWS_REGION': boto3.Session().region_name,
}
custom_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args)
print("Using ECR image %s" % custom_image_name)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Logged into ECR
Building docker image sagemaker-roboschool-ray-cpu from Dockerfile
$ docker build -t sagemaker-roboschool-ray-cpu -f Dockerfile . --build-arg CPU_OR_GPU=cpu --build-arg AWS_REGION=us-west-2
Sending build context to Docker daemon  688.1kB
Step 1/14 : ARG CPU_OR_GPU
Step 2/14 : ARG AWS_REGION
Step 3/14 : FROM 520713654638.dkr.ecr.${AWS_REGION}.amazonaws.com/sagemaker-rl-tensorflow:ray0.6.5-${CPU_OR_GPU}-py3
ray0.6.5-cpu-py3: Pulling from sagemaker-rl-tensorflow
7e6591854262: Pulling fs layer
089d60cb4e0a: Pulling fs layer
9c461696bc09: Pulling fs layer
45085432511a: Pulling fs layer
133ef7c773de: Pulling fs layer
536fa53fdaf4: Pulling fs layer
9b060350118c: Pulli

1 upgraded, 126 newly installed, 0 to remove and 93 not upgraded.
Need to get 31.1 MB of archives.
After this operation, 162 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu xenial/main amd64 libpopt0 amd64 1.16-10 [26.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 cmake-data all 3.5.1-1ubuntu3 [1121 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial/main amd64 liblzo2-2 amd64 2.08-1.2 [48.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 zlib1g amd64 1:1.2.8.dfsg-2ubuntu4.3 [51.2 kB]
Get:5 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libarchive13 amd64 3.1.2-11ubuntu0.16.04.8 [262 kB]
Get:6 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libcurl3 amd64 7.47.0-1ubuntu2.14 [186 kB]
Get:7 http://archive.ubuntu.com/ubuntu xenial/main amd64 libjsoncpp1 amd64 1.7.2-1 [73.0 kB]
Get:8 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 cmake amd64 3.5.1-1ubuntu3 [2623 kB]
Get:9 http://a

Get:83 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxcb-shape0-dev amd64 1.11.1-1ubuntu1 [6900 B]
Get:84 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxcb-xfixes0-dev amd64 1.11.1-1ubuntu1 [11.2 kB]
Get:85 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxcb-sync-dev amd64 1.11.1-1ubuntu1 [10.1 kB]
Get:86 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxcb-present-dev amd64 1.11.1-1ubuntu1 [6618 B]
Get:87 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxshmfence-dev amd64 1.2-1 [3676 B]
Get:88 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libx11-xcb-dev amd64 2:1.6.3-1ubuntu2.1 [9718 B]
Get:89 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libwayland-cursor0 amd64 1.12.0-1~ubuntu16.04.3 [10.1 kB]
Get:90 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libwayland-bin amd64 1.12.0-1~ubuntu16.04.3 [18.4 kB]
Get:91 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libwayland-dev amd64 1.12.0-1~ubuntu16.04.3 [92.4

Selecting previously unselected package libmircore1:amd64.
Preparing to unpack .../libmircore1_0.26.3+16.04.20170605-0ubuntu1.1_amd64.deb ...
Unpacking libmircore1:amd64 (0.26.3+16.04.20170605-0ubuntu1.1) ...
Selecting previously unselected package libmircommon7:amd64.
Preparing to unpack .../libmircommon7_0.26.3+16.04.20170605-0ubuntu1.1_amd64.deb ...
Unpacking libmircommon7:amd64 (0.26.3+16.04.20170605-0ubuntu1.1) ...
Selecting previously unselected package libprotobuf-lite9v5:amd64.
Preparing to unpack .../libprotobuf-lite9v5_2.6.1-1.3_amd64.deb ...
Unpacking libprotobuf-lite9v5:amd64 (2.6.1-1.3) ...
Selecting previously unselected package libmirprotobuf3:amd64.
Preparing to unpack .../libmirprotobuf3_0.26.3+16.04.20170605-0ubuntu1.1_amd64.deb ...
Unpacking libmirprotobuf3:amd64 (0.26.3+16.04.20170605-0ubuntu1.1) ...
Selecting previously unselected package libxkbcommon0:amd64.
Preparing to unpack .../libxkbcommon0_0.5.0-1ubuntu2.1_amd64.deb ...
Unpacking libxkbcommon0:amd64 (0.5.0-1

Selecting previously unselected package libxdmcp-dev:amd64.
Preparing to unpack .../libxdmcp-dev_1%3a1.1.2-1.1_amd64.deb ...
Unpacking libxdmcp-dev:amd64 (1:1.1.2-1.1) ...
Selecting previously unselected package x11proto-input-dev.
Preparing to unpack .../x11proto-input-dev_2.3.1-1_all.deb ...
Unpacking x11proto-input-dev (2.3.1-1) ...
Selecting previously unselected package x11proto-kb-dev.
Preparing to unpack .../x11proto-kb-dev_1.0.7-0ubuntu1_all.deb ...
Unpacking x11proto-kb-dev (1.0.7-0ubuntu1) ...
Selecting previously unselected package xtrans-dev.
Preparing to unpack .../xtrans-dev_1.3.5-1_all.deb ...
Unpacking xtrans-dev (1.3.5-1) ...
Selecting previously unselected package libpthread-stubs0-dev:amd64.
Preparing to unpack .../libpthread-stubs0-dev_0.3-4_amd64.deb ...
Unpacking libpthread-stubs0-dev:amd64 (0.3-4) ...
Selecting previously unselected package libxcb1-dev:amd64.
Preparing to unpack .../libxcb1-dev_1.11.1-1ubuntu1_amd64.deb ...
Unpacking libxcb1-dev:amd64 (1.11.1-1ub

Selecting previously unselected package libqt5concurrent5:amd64.
Preparing to unpack .../libqt5concurrent5_5.5.1+dfsg-16ubuntu7.7_amd64.deb ...
Unpacking libqt5concurrent5:amd64 (5.5.1+dfsg-16ubuntu7.7) ...
Selecting previously unselected package libqt5opengl5:amd64.
Preparing to unpack .../libqt5opengl5_5.5.1+dfsg-16ubuntu7.7_amd64.deb ...
Unpacking libqt5opengl5:amd64 (5.5.1+dfsg-16ubuntu7.7) ...
Selecting previously unselected package libqt5printsupport5:amd64.
Preparing to unpack .../libqt5printsupport5_5.5.1+dfsg-16ubuntu7.7_amd64.deb ...
Unpacking libqt5printsupport5:amd64 (5.5.1+dfsg-16ubuntu7.7) ...
Selecting previously unselected package libqt5sql5:amd64.
Preparing to unpack .../libqt5sql5_5.5.1+dfsg-16ubuntu7.7_amd64.deb ...
Unpacking libqt5sql5:amd64 (5.5.1+dfsg-16ubuntu7.7) ...
Selecting previously unselected package libqt5test5:amd64.
Preparing to unpack .../libqt5test5_5.5.1+dfsg-16ubuntu7.7_amd64.deb ...
Unpacking libqt5test5:amd64 (5.5.1+dfsg-16ubuntu7.7) ...
Selecting 

Setting up libxdamage-dev:amd64 (1:1.1.4-2) ...
Setting up libxcb-glx0-dev:amd64 (1.11.1-1ubuntu1) ...
Setting up libxcb-dri2-0-dev:amd64 (1.11.1-1ubuntu1) ...
Setting up libxcb-dri3-dev:amd64 (1.11.1-1ubuntu1) ...
Setting up libxcb-render0-dev:amd64 (1.11.1-1ubuntu1) ...
Setting up libxcb-randr0-dev:amd64 (1.11.1-1ubuntu1) ...
Setting up libxcb-shape0-dev:amd64 (1.11.1-1ubuntu1) ...
Setting up libxcb-xfixes0-dev:amd64 (1.11.1-1ubuntu1) ...
Setting up libxcb-sync-dev:amd64 (1.11.1-1ubuntu1) ...
Setting up libxcb-present-dev:amd64 (1.11.1-1ubuntu1) ...
Setting up libxshmfence-dev:amd64 (1.2-1) ...
Setting up libx11-xcb-dev:amd64 (2:1.6.3-1ubuntu2.1) ...
Setting up libwayland-cursor0:amd64 (1.12.0-1~ubuntu16.04.3) ...
Setting up libwayland-bin (1.12.0-1~ubuntu16.04.3) ...
Setting up libwayland-dev:amd64 (1.12.0-1~ubuntu16.04.3) ...
Setting up libmircore-dev:amd64 (0.26.3+16.04.20170605-0ubuntu1.1) ...
Setting up zlib1g-dev:amd64 (1:1.2.8.dfsg-2ubuntu4.3) ...
Setting up libprotobuf9v5:amd

Get:16 http://archive.ubuntu.com/ubuntu xenial/universe amd64 libboost-python-dev amd64 1.58.0.1ubuntu1 [3170 B]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 39.5 MB in 11s (3525 kB/s)
Selecting previously unselected package libpython2.7-minimal:amd64.
(Reading database ... 24523 files and directories currently installed.)
Preparing to unpack .../libpython2.7-minimal_2.7.12-1ubuntu0~16.04.9_amd64.deb ...
Unpacking libpython2.7-minimal:amd64 (2.7.12-1ubuntu0~16.04.9) ...
Selecting previously unselected package python2.7-minimal.
Preparing to unpack .../python2.7-minimal_2.7.12-1ubuntu0~16.04.9_amd64.deb ...
Unpacking python2.7-minimal (2.7.12-1ubuntu0~16.04.9) ...
Selecting previously unselected package python-minimal.
Preparing to unpack .../python-minimal_2.7.12-1~16.04_amd64.deb ...
Unpacking python-minimal (2.7.12-1~16.04) ...
Selecting previously unselected package libpython2.7-stdlib:amd64.
Preparing to unpack .../libpython2.7-stdlib_2.7.12-1ub

Setting up libpython3.6:amd64 (3.6.10-1+xenial1) ...
Setting up libpython3.6-dev:amd64 (3.6.10-1+xenial1) ...
Setting up python3.6-dev (3.6.10-1+xenial1) ...
Processing triggers for libc-bin (2.23-0ubuntu11) ...
Removing intermediate container d5e176fd6cfa
 ---> 74c18ebaa630
Step 8/14 : RUN curl -fSsL -O https://bootstrap.pypa.io/get-pip.py &&     python get-pip.py &&     rm get-pip.py
 ---> Running in 68f7959dc37b
  Attempting uninstall: pip
    Found existing installation: pip 18.1
    Uninstalling pip-18.1:
      Successfully uninstalled pip-18.1
Removing intermediate container 68f7959dc37b
 ---> 3262d08cd78d
Step 9/14 : RUN pip install --upgrade     pip     setuptools     setproctitle     lz4     psutil
 ---> Running in 1812dcb5c378
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.0.2)
  Created wheel for setproctitle: filename=setproctitle-1.1.10-cp36-cp36m-linux_x86_64.whl size=26441 sha256=3427d1843d7fc8e783dc7fd73bff4d8b736c4f8806e626cea2938ed26

  Created wheel for sagemaker-containers: filename=sagemaker_containers-2.8.3-cp36-cp36m-linux_x86_64.whl size=77894 sha256=7e823c0913f15eba67d9d669506f973a3cd081895bc59059b34867725143ef16
  Stored in directory: /root/.cache/pip/wheels/16/f6/f7/9179c22d15775709edf3dd990067cb1b2a171a29b0727d9393
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=e1e6409db14eb518678f39d277048fb1908c1bb730954da4171b1b26d6856f2d
  Stored in directory: /root/.cache/pip/wheels/ac/cb/8a/b27bf6323e2f4c462dcbf77d70b7c5e7868a7fbe12871770cf
  Created wheel for inotify-simple: filename=inotify_simple-1.2.1-py3-none-any.whl size=8203 sha256=35fa9c56a663bebbb90dbdc8b92f6c2e24ba3efccd94d92e7a18a05f5f76b1d1
  Stored in directory: /root/.cache/pip/wheels/a5/34/f6/5d4eae9a326dc637344cd68353ddd119e9f6f575e925502888
  Created wheel for psutil: filename=psutil-5.6.7-cp36-cp36m-linux_x86_64.whl size=258592 sha256=a5c1fcda72cfbca9416092eb6e7d1e4031fb71a5e190ee840f38c15d1618f8da
  Stored 

## 튜닝 구성

튜닝 작업은 훌륭한 구성을 찾을 때 무엇을 변화시켜야 하는지 알아야 합니다. 때때로, 이를 검색 공간(search space) 또는 구성 공간(configuration space)이라고 합니다. 이를 위해, 하이퍼파라미터 셋을 선택하여 각 범위를 지정합니다. 하이퍼파라미터를 자동으로 튜닝해 본 경험이 없으면, 한 번에 하나 또는 두 개의 하이퍼파라미터로 시작해 보면서 결과에 어떤 영향을 미치는지 확인해야 합니다.

In [8]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# The hyperparameters we're going to tune
hyperparameter_ranges = {
    # inspired by https://medium.com/aureliantactics/ppo-hyperparameters-and-ranges-6fc2d29bccbe
    #'rl.training.config.clip_param': ContinuousParameter(0.1, 0.4),
    #'rl.training.config.kl_target': ContinuousParameter(0.003, 0.03),
    #'rl.training.config.vf_loss_coeff': ContinuousParameter(0.5, 1.0),
    #'rl.training.config.entropy_coeff': ContinuousParameter(0.0, 0.01),
    'rl.training.config.kl_coeff': ContinuousParameter(0.5, 1.0),
    'rl.training.config.num_sgd_iter': IntegerParameter(3, 50),
}

# The hyperparameters that are the same for all jobs
static_hyperparameters = {
    "rl.training.stop.time_total_s": 600,  # Tell each training job to stop after 10 minutes
    #'rl.training.config.num_sgd_iter': 7,
    #'rl.training.config.sgd_minibatch_size': 1000,
    #'rl.training.config.train_batch_size': 25000,
}

## 튜닝 작업을 시작할 준비 수행

먼저 단일 훈련 작업을 시작하는 것처럼 estimator를 생성합니다. 이는 `tuner` 객체를 만드는 데 사용됩니다.

In [9]:
metric_definitions = RLEstimator.default_metric_definitions(RLToolkit.RAY)
estimator = RLEstimator(entry_point="train-%s.py" % roboschool_problem,
                        source_dir='src',
                        dependencies=["common/sagemaker_rl"],
                        image_name=custom_image_name,
                        role=role,
                        train_instance_type=instance_type,
                        train_instance_count=1,
                        output_path=s3_output_path,
                        base_job_name=job_name_prefix,
                        metric_definitions=metric_definitions,
                        hyperparameters=static_hyperparameters,
                    )

tuner = HyperparameterTuner(estimator,
                            objective_metric_name='episode_reward_mean',
                            objective_type='Maximize',
                            hyperparameter_ranges=hyperparameter_ranges,
                            metric_definitions=metric_definitions,
                            max_jobs=max_jobs,
                            max_parallel_jobs=max_parallel_jobs,
                            base_tuning_job_name=job_name_prefix,
                           )
tuner.fit()

## 진행 상황 모니터링

튜닝 작업이 수행되는 방식을 보려면 SageMaker 콘솔로 이동하세요. **Training** 섹션에는 하이퍼파라미터 튜닝 작업이 표시되며, 여기서 새로 만든 작업이 표시됩니다. 계정에서 일련의 `TrainingJobs`가 시작되며 각각의 정규 훈련 작업처럼 작동합니다. 각 작업이 완료되면 평균 보상에 대해 달성한 최종값이 표시됩니다. 각 작업은 또한 CloudWatch에 알고리즘 지표(metric)를 내보내며 CloudWatch 지표에 표시됩니다. 이를 보려면, 훈련 작업을 클릭하여 세부 정보 페이지로 이동한 다음 "View algorithm metrics" 링크를 찾아 해당 작업이 어떻게 진행되고 있는지 차트를 볼 수 있습니다. 또한CloudWatch 콘솔에서 검색 기준을 변경하면, 본 튜닝 작업의 모든 작업들에 대한 지표를 오버레이할 수 있습니다.